# Project - Apache Spark & Elastichsearch

##### Students:
* Lilia IZRI      (DS)
* Yacine MOKHTARI (DS)
* Alexandre COMBEAU (DS)

##### Report
[PENSER A METTRE UN LIEN ICI]


In [2]:
# !pip install textblob
!pip install elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.6/378.6 KB 10.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 KB 6.0 MB/s eta 0:00:00


In [1]:
# import necessary packages
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.streaming import StreamingContext
from textblob import TextBlob

# For ML
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.clustering import StreamingKMeans

In [2]:
# Initiate the SparkContext and StreamingContext with 10 second batch interval
sc = SparkContext()
ssc = StreamingContext(sc, 10)
spark = SparkSession(sc)
ssc.checkpoint("file:///tmp/spark")    # Checkpoint for backups (useful for operations by window)

## I. Process & Analyze input data (tweets)
### 1. Create our Dstream that receives data

In [3]:
# initiate streaming text from a TCP (socket) source (Our tweets received)
socket_stream = ssc.socketTextStream("127.0.0.1", 5567)

### 2. Process data and tag with sentiment 

In [4]:
# We create a function that analysis text with textblob
def sentiment(text):
    """ Function that returns -1 if a tweet is more likely negative (polarity<0)
                               0 if it's neutral  (polarity==0)
                               1 if it's positive (polarity>0)
    """
    polarity = TextBlob(text).polarity
    return 1 if polarity > 0 else -1 if polarity < 0 else 0

Here, we just took into account the polarity and choosed to ignore the subjectivity !  ;)

In [5]:
# We split the fields of the tweet received and we add tag the data with the sentiment of the tweet
#   so the rdd below 'tweets' will be of the form (user, text, date, location, hashtags, sentiment)

def mapSplit(tweet):
    """
    A function that takes a tweet  (the one we sent from the other iPython file),
    splits it into its different fields and adds the sentiment field {-1, 0, 1}
    """
    return (tweet[1],    tweet[2],  tweet[3],    tweet[4],    tweet[5],    tweet[6],   sentiment(tweet[2][6:]), tweet[7])
             #user        #text       #date      #latitude   #longitude    #Hashtags    #sentiment(= {-1,0,1})   #Id
    
tweets_split = socket_stream.map(lambda tweet: tweet.split(' ###:field:### '))
tweets = tweets_split.map(mapSplit)
tweets.pprint()

Form of RDD in Tweets :
``('user: userX', 'tweet: @userY blablablbabla', 'date: Thu May 05 00:16:06 +0000 2022', 'lat: 44.933143', 'lon: 7.540121', 'hashtags: #SaveTheWorld', 0, "1237288393929")``

### 3. ML : Cluster tweets according to sentiments and their location

In [6]:
# We create a training set and test set 
training_data =  tweets.map(lambda tweet: Vectors.dense([float(tweet[6]), float(tweet[3][5:]), float(tweet[4][5:])]))
testing_data  =  tweets.map(lambda tweet: LabeledPoint(float(tweet[6]), Vectors.dense([float(tweet[6]), float(tweet[3][5:]), float(tweet[4][5:])])))


# We create a model with random clusters and specify the number of clusters to find
k = 3
dimension = 3
weights = 0.0
seed = 21

# init
model = StreamingKMeans(k=k, decayFactor=0.5).setRandomCenters(dimension, weights, seed)

# Train the model
model.trainOn(training_data)  

# Predict
result = model.predictOnValues(testing_data.map(lambda lp: (lp.label, lp.features)))
result.pprint()

In [7]:
# We keep the predictions of each tweet (the index of the cluster), and we create (indexCluster, 1) pairs
predictions   = result.map(lambda x: (x[1], 1))

# We reduce by key and window to get the number of elements assigned to each cluster
size_clusters = predictions.reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 30, 10)
size_clusters.pprint()

In [9]:
tweets.saveAsTextFiles("tmp/")

In [10]:
# start streaming and wait couple of minutes to get enought tweets
ssc.start()

-------------------------------------------
Time: 2022-05-05 13:52:00
-------------------------------------------
('user: MyGrindelwald', "tweet: RT @sunflxwervolsix: Johnny Depp fought for this trial to be public for a reason and now that you've seen Amber Heard's less than desirable…", 'date: Thu May 05 13:51:47 +0000 2022', 'lat: 51.096246199999996', 'lon: 4.178629103169916', 'hashtags: ', -1, 'id: 4583281223')

-------------------------------------------
Time: 2022-05-05 13:52:00
-------------------------------------------
(-1.0, 1)

-------------------------------------------
Time: 2022-05-05 13:52:00
-------------------------------------------
(1, 1)

-------------------------------------------
Time: 2022-05-05 13:52:10
-------------------------------------------
('user: kimkim32701', 'tweet: Why Johnny Depp And Winona Ryder Never Ended Up Together | Rumour Juice https://t.co/n6eG7pc6K9 via @YouTube', 'date: Thu May 05 13:51:47 +0000 2022', 'lat: 41.9053703', 'lon: -73.976249', '

In [11]:
print("Clusters coordinates: " + str(model.latestModel().clusterCenters))

Clusters coordinates: [[-0.05196425 -0.11119605  1.0417968 ]
 [-1.25673929  0.74538768 -1.71105376]
 [-0.20586438 -0.23457129  1.12814404]]
-------------------------------------------
Time: 2022-05-05 13:40:50
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 13:40:50
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 13:40:50
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 13:41:00
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 13:41:00
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 13:41:00
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 13:41:10
-------------------------------------------
('user: Pamitsmyname', 'tweet: RT @johndeppwarrior: Don’

In [11]:
print("Clusters coordinates: " + str(model.latestModel().clusterCenters))

Clusters coordinates: [[-0.05196425 -0.11119605  1.0417968 ]
 [-1.25673929  0.74538768 -1.71105376]
 [-0.20586438 -0.23457129  1.12814404]]
-------------------------------------------
Time: 2022-05-05 13:20:00
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 13:20:10
-------------------------------------------
('user: sefluctuat', 'tweet: 💥💥💥 Appel à des experts juristes : Comment destituer Macron?', 'date: Thu May 05 13:19:56 +0000 2022', 'lat: 44.933143', 'lon: 7.540121', 'hashtags: ', 0, 'id: 1420734268062539783')
('user: cryptoMarseille', "tweet: RT @PoncetRaymonde: Ces centaines de gestes désintéressés, au service de l'intérêt général,  prouvent que nous avons raison de penser qu'un…", 'date: Thu May 05 13:19:56 +0000 2022', 'lat: 25.029422', 'lon: -77.36195598496681', 'hashtags: ', 0, 'id: 1465079661852073994')

-------------------------------------------
Time: 2022-05-05 13:20:10
-------------------------------------------

In [3]:
## La bizarrement ça marche tout seul... mais quand c'est un stream ça plante


# a = "id: 234774849593"

# es.index(index="hello",
#                 id=int(a[4:]),
#                 document={"user": "lili",
#                           "text": "22"})